# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [4]:
# all import statements needed for the project, for example:

import os
import re
from bs4 import BeautifulSoup
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db

In [81]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "weather_data"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [6]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones
In this section, we loaded the taxi shapefile that corresponds location IDs to geographical latitudes and logitudes. 
* The `load_taxi_zones` function reads the shapefile and use GeoPandas to read the file
* The `lookup_coords_for_taxi_zone_id` function takes location IDs and the loaded shapefile and returns a tuple of latitude and logitude
* The `make_loc_id_coords_dict` creates a dictionary of location IDs and coordinates that will be used in data cleaning

In [12]:
def load_taxi_zones(shapefile):
    taxi_zones = gpd.read_file(shapefile)
    return taxi_zones

In [14]:
taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
taxi_zones.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US survey foot)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [16]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones=taxi_zones):
    zone = loaded_taxi_zones[loaded_taxi_zones["LocationID"] == zone_loc_id]
    centroid = zone.geometry.centroid.iloc[0]
    centroid_geo = gpd.GeoSeries([centroid], crs=loaded_taxi_zones.crs).to_crs(epsg=CRS).iloc[0]

    latitude = centroid_geo.y
    longitude = centroid_geo.x

    return (latitude, longitude)

In [18]:
def make_loc_id_coords_dict(loaded_taxi_zones):
    id_coords_dict = {}
    for loc_id in loaded_taxi_zones["LocationID"]:
        id_coords_dict[loc_id] = lookup_coords_for_taxi_zone_id(loc_id, loaded_taxi_zones)

    return id_coords_dict

In [20]:
ID_COORDS_DICT = make_loc_id_coords_dict(taxi_zones)

### Calculate Sample Size

In [22]:
def calculate_sample_size(population):
    confidence_level = 0.95
    margin_of_error = 0.05
    proportion = 0.5
    
    from scipy.stats import norm

    z_score = norm.ppf(1 - (1 - confidence_level) / 2)

    # Cochran’s
    n = (z_score**2 * proportion * (1 - proportion)) / (margin_of_error**2)
    
    # Adjust for finite population
    n_adj = n / (1 + (n - 1) / population)
    
    return int(round(n_adj)) 

### Common Functions
* `get_all_urls_from_taxi_page` fetches information on the taxi page and finds all "Yellow Taxi Trip Records" urls and "High Volume For-Hire Vehicle Trip Records" urls
* `find_parquet_urls` uses regex to filter the urls that ends with ".parquet" to make sure that the urls are parquet files
* `download_parquet` creates a directory and downloads relevant parquets into the directory
* `get_and_clean_month` filters the urls that are from January 2020 to August 2024
* `sample_monthly` function reads all the parquet files in a directory, finds the file with largest number of rows and computes the sample size using the "maximum population". Next it creates samples for all files using the computed sample size and combine them into a single dataframe.

In [26]:
def get_all_urls_from_taxi_page(taxi_page):
    response = requests.get(taxi_page)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    yellow_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    fhvhv_tags = soup.find_all("a", attrs={"title": "High Volume For-Hire Vehicle Trip Records"})

    yellow_urls = [a["href"].strip() for a in yellow_tags]
    fhvhv_urls = [a["href"].strip() for a in fhvhv_tags]
    
    return yellow_urls, fhvhv_urls

In [28]:
def find_parquet_urls(urls):
    pattern = re.compile(r"\.parquet$")
    parquet_urls = [url for url in urls if pattern.search(url)]    
    return parquet_urls

In [30]:
def download_parquet(urls, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for url in urls:
        filename = os.path.basename(url)
        output_path = os.path.join(output_dir, filename)
        if os.path.exists(output_path):
            continue        
        response = requests.get(url, stream=True)
        with open(output_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
        print(f"Downloaded {filename} to {output_dir}")

In [32]:
def get_and_clean_month(urls):
    pattern = re.compile(r"(202[0-3]-(0[1-9]|1[0-2])|2024-(0[1-8]))")
    cleaned_urls = [url for url in urls if pattern.search(url)]
    return cleaned_urls

In [34]:
def read_parquet_to_df(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".parquet")]
    all_dataframe = []
    for file in files:
        
        df = pd.read_parquet(file)
        all_dataframe.append(df)
    if all_dataframe:
        combined_df = pd.concat(all_dataframe, ignore_index=True)
        return combined_df

In [36]:
def clean_parquet_column(file_path, columns_to_keep):
    df = pd.read_parquet(file_path)
    cleaned_df = df[columns_to_keep]
    return cleaned_df

In [38]:
yellow_urls, fhvhv_urls = get_all_urls_from_taxi_page(TLC_URL)
taxi_parquet = find_parquet_urls(yellow_urls)
uber_parquet = find_parquet_urls(fhvhv_urls)
taxi_urls = get_and_clean_month(taxi_parquet)
uber_urls = get_and_clean_month(uber_parquet)

In [40]:
def sample_monthly(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.parquet')]
    
    max_rows = 0
    for file in files:
        df = pd.read_parquet(file)
        max_rows = max(max_rows, len(df))

    sample_size = calculate_sample_size(max_rows)
    print(f"Sample size for all months: {sample_size}")

    sampled_dataframes = []
    for file in files:
        df = pd.read_parquet(file)
        sampled_df = df.sample(n=sample_size, random_state=30, replace=False)
        sampled_dataframes.append(sampled_df)
    print("Finished sampling")

    if sampled_dataframes:
        combined_sampled_df = pd.concat(sampled_dataframes, ignore_index=True)
        return combined_sampled_df

### Process Taxi Data

In [43]:
taxi_data_dir = "taxi_data"
download_parquet(taxi_urls, taxi_data_dir)

Downloaded yellow_tripdata_2024-01.parquet to taxi_data
Downloaded yellow_tripdata_2024-02.parquet to taxi_data
Downloaded yellow_tripdata_2024-03.parquet to taxi_data
Downloaded yellow_tripdata_2024-04.parquet to taxi_data
Downloaded yellow_tripdata_2024-05.parquet to taxi_data
Downloaded yellow_tripdata_2024-06.parquet to taxi_data
Downloaded yellow_tripdata_2024-07.parquet to taxi_data
Downloaded yellow_tripdata_2024-08.parquet to taxi_data
Downloaded yellow_tripdata_2023-01.parquet to taxi_data
Downloaded yellow_tripdata_2023-02.parquet to taxi_data
Downloaded yellow_tripdata_2023-03.parquet to taxi_data
Downloaded yellow_tripdata_2023-04.parquet to taxi_data
Downloaded yellow_tripdata_2023-05.parquet to taxi_data
Downloaded yellow_tripdata_2023-06.parquet to taxi_data
Downloaded yellow_tripdata_2023-07.parquet to taxi_data
Downloaded yellow_tripdata_2023-08.parquet to taxi_data
Downloaded yellow_tripdata_2023-09.parquet to taxi_data
Downloaded yellow_tripdata_2023-10.parquet to ta

In [45]:
sampled_taxi_df = sample_monthly(taxi_data_dir)

Sample size for all months: 384
Finished sampling


C:\Users\NIU\AppData\Local\Temp\ipykernel_5408\1458695063.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_sampled_df = pd.concat(sampled_dataframes, ignore_index=True)


In [47]:
sampled_taxi_df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
0,2,2020-01-21 15:41:03,2020-01-21 15:54:32,2.0,0.71,1.0,N,233,164,1,9.0,0.0,0.5,1.84,0.0,0.3,14.14,2.5,NaN,NaN
1,2,2020-01-29 21:06:16,2020-01-29 21:11:34,1.0,0.76,1.0,N,141,140,2,5.5,0.5,0.5,0.00,0.0,0.3,9.30,2.5,NaN,NaN
2,2,2020-01-26 20:19:48,2020-01-26 20:25:39,3.0,0.77,1.0,N,161,163,2,5.5,0.5,0.5,0.00,0.0,0.3,9.30,2.5,NaN,NaN
3,2,2020-01-16 13:17:34,2020-01-16 13:24:01,1.0,0.99,1.0,N,143,239,1,6.5,0.0,0.5,1.96,0.0,0.3,11.76,2.5,NaN,NaN
4,2,2020-01-30 20:20:32,2020-01-30 20:28:30,1.0,0.84,1.0,N,233,162,1,6.5,0.5,0.5,1.20,0.0,0.3,11.50,2.5,NaN,NaN
5,2,2020-01-29 19:21:58,2020-01-29 19:35:52,1.0,1.54,1.0,N,90,161,1,10.0,1.0,0.5,3.58,0.0,0.3,17.88,2.5,NaN,NaN
6,1,2020-01-12 17:42:09,2020-01-12 17:47:26,2.0,1.10,1.0,N,140,236,2,6.0,2.5,0.5,0.00,0.0,0.3,9.30,2.5,NaN,NaN
7,1,2020-01-29 22:03:48,2020-01-29 22:08:19,1.0,0.90,1.0,N,142,239,2,5.5,3.0,0.5,0.00,0.0,0.3,9.30,2.5,NaN,NaN
8,2,2020-01-21 21:17:23,2020-01-21 21:36:29,4.0,3.78,1.0,N,161,148,1,15.0,0.5,0.5,3.76,0.0,0.3,22.56,2.5,NaN,NaN
9,1,2020-01-15 01:27:14,2020-01-15 01:32:45,1.0,0.90,1.0,N,161,233,1,6.0,3.0,0.5,2.90,0.0,0.3,12.70,2.5,NaN,NaN


In [63]:
def get_and_clean_taxidata(dataframe):
    try: 
        print(f"Cleaning the sample dataframe...")

        if not isinstance(dataframe, pd.DataFrame):
            raise ValueError("must Pandas DataFrame")

        # look up the latitude and longitude (get those coordinates)
        dataframe[["latitude_pickup", "longitude_pickup"]] = dataframe["PULocationID"].map(ID_COORDS_DICT).apply(pd.Series)
        dataframe[["latitude_dropoff", "longitude_dropoff"]] = dataframe["DOLocationID"].map(ID_COORDS_DICT).apply(pd.Series)

        
        # remove some location IDs not valid and distance is 0
        dataframe = dataframe.dropna(subset=['latitude_pickup', 'longitude_pickup', 'latitude_dropoff', 'longitude_dropoff'])
        dataframe = dataframe[dataframe["trip_distance"] != 0]


        # remove unnecessary columns
        columns_to_keep = [
            'tpep_pickup_datetime', 'tpep_dropoff_datetime',
            'trip_distance', 
            'latitude_pickup', 'longitude_pickup', 'latitude_dropoff', 'longitude_dropoff' , 
            'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee', 
            'total_amount'
        ]         #调整
        dataframe = dataframe[columns_to_keep]


        # normalize column names
        dataframe.columns = [col.lower().replace(' ', '_') for col in dataframe.columns]

        # normalizing and using appropriate column types for the respective data;
        dataframe['tpep_pickup_datetime'] = pd.to_datetime(dataframe['tpep_pickup_datetime'])
        dataframe['tpep_dropoff_datetime'] = pd.to_datetime(dataframe['tpep_dropoff_datetime'])
        dataframe['trip_distance'] = dataframe['trip_distance'].astype(float)

        # for Yellow Taxi data, remove trips that start and/or end outside of  (40.560445, -74.242330) and (40.908524, -73.717047) ie NEW_YORK_BOX_COORDS.
        lat_min, lon_min = NEW_YORK_BOX_COORDS[0]
        lat_max, lon_max = NEW_YORK_BOX_COORDS[1]
        
        dataframe = dataframe[
            (dataframe['latitude_pickup'].between(lat_min, lat_max)) &
            (dataframe['longitude_pickup'].between(lon_min, lon_max)) &
            (dataframe['latitude_dropoff'].between(lat_min, lat_max)) &
            (dataframe['longitude_dropoff'].between(lon_min, lon_max))
        ]

       
        
        return dataframe

    except Exception as e:
        print(f"Error processing the dataframe: {e}")
        return None

In [65]:
taxi_data = get_and_clean_taxidata(sampled_taxi_df)

Cleaning the sample dataframe...


In [67]:
taxi_data.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,latitude_pickup,longitude_pickup,latitude_dropoff,longitude_dropoff,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,airport_fee,total_amount
0,2020-01-21 15:41:03,2020-01-21 15:54:32,0.71,40.749914,-73.970443,40.748575,-73.985156,9.0,0.0,0.5,1.84,0.0,0.3,2.5,NaN,14.14
1,2020-01-29 21:06:16,2020-01-29 21:11:34,0.76,40.766948,-73.959635,40.765484,-73.954739,5.5,0.5,0.5,0.00,0.0,0.3,2.5,NaN,9.30
2,2020-01-26 20:19:48,2020-01-26 20:25:39,0.77,40.758028,-73.977698,40.764421,-73.977569,5.5,0.5,0.5,0.00,0.0,0.3,2.5,NaN,9.30
3,2020-01-16 13:17:34,2020-01-16 13:24:01,0.99,40.775965,-73.987646,40.783961,-73.978632,6.5,0.0,0.5,1.96,0.0,0.3,2.5,NaN,11.76
4,2020-01-30 20:20:32,2020-01-30 20:28:30,0.84,40.749914,-73.970443,40.756688,-73.972356,6.5,0.5,0.5,1.20,0.0,0.3,2.5,NaN,11.50


In [69]:
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20869 entries, 0 to 21503
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tpep_pickup_datetime   20869 non-null  datetime64[us]
 1   tpep_dropoff_datetime  20869 non-null  datetime64[us]
 2   trip_distance          20869 non-null  float64       
 3   latitude_pickup        20869 non-null  float64       
 4   longitude_pickup       20869 non-null  float64       
 5   latitude_dropoff       20869 non-null  float64       
 6   longitude_dropoff      20869 non-null  float64       
 7   fare_amount            20869 non-null  float64       
 8   extra                  20869 non-null  float64       
 9   mta_tax                20869 non-null  float64       
 10  tip_amount             20869 non-null  float64       
 11  tolls_amount           20869 non-null  float64       
 12  improvement_surcharge  20869 non-null  float64       
 13  conges

In [71]:
taxi_data.describe()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,latitude_pickup,longitude_pickup,latitude_dropoff,longitude_dropoff,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,airport_fee,total_amount
count,20869,20869,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,19870.000000,7930.000000,20869.000000
mean,2022-05-01 23:23:35.520628,2022-05-01 23:40:27.647515,3.313074,40.753511,-73.966552,40.755856,-73.970556,15.666208,1.228502,0.490718,2.726723,0.456978,0.544477,2.293281,0.088115,22.794868
min,2020-01-01 00:20:28,2020-01-01 00:33:35,0.010000,40.576961,-74.167234,40.576961,-74.174002,-171.700000,-5.000000,-0.500000,0.000000,-15.380000,-1.000000,-2.500000,-1.250000,-188.080000
25%,2021-02-28 14:22:00,2021-02-28 14:42:00,1.090000,40.740439,-73.989845,40.740337,-73.989845,7.200000,0.000000,0.500000,0.000000,0.000000,0.300000,2.500000,0.000000,12.600000
50%,2022-04-30 10:32:36,2022-04-30 10:44:33,1.810000,40.758028,-73.977698,40.758028,-73.977698,10.700000,0.500000,0.500000,2.160000,0.000000,0.300000,2.500000,0.000000,16.800000
75%,2023-07-01 09:11:58,2023-07-01 09:26:38,3.400000,40.773633,-73.961764,40.775932,-73.959635,17.500000,2.500000,0.500000,3.500000,0.000000,1.000000,2.500000,0.000000,24.500000
max,2024-08-31 23:35:57,2024-08-31 23:44:08,55.600000,40.897932,-73.735554,40.899528,-73.726655,7000.500000,11.750000,0.800000,33.440000,40.000000,1.000000,2.500000,1.250000,7010.850000
std,NaN,NaN,4.135789,0.032516,0.045546,0.033581,0.037030,50.405868,1.513872,0.088052,3.153968,1.878248,0.350941,0.738788,0.321830,51.654328


### Processing Uber Data
This section downloads the relevant parquet files from the taxi website and creates a sample according to the sampling function of the Uber data. The sample dataframe is cleaned using the `get_and_clean_uber_data` function.

* The `filter_uber_and_sample_monthly` function uses similar logic as the `sample_monthly` function, but it filters Uber trips before sampling

* reads parquet files in the directory and filter Uber data each month, then creates a sample of each month and integrate the sample datasets into one dataset.

* The `get_and_clean_uber_data` function takes a dataframe and returns a cleaned dataframe that:
    * Filtered Uber rides
    * Converted Location IDs to latitude lognitude coordinates
    * Computed total fares for each ride
    * Filtered rides that start and/or end within the New York bounding box
    * Dropped columns that are irrelevant to later parts of the project
    * Normalized column names and removed invalid data

In [ ]:
uber_data_dir = "uber_data"
download_parquet(uber_urls, uber_data_dir)

Downloaded fhvhv_tripdata_2024-01.parquet to uber_data
Downloaded fhvhv_tripdata_2024-02.parquet to uber_data
Downloaded fhvhv_tripdata_2024-03.parquet to uber_data
Downloaded fhvhv_tripdata_2024-04.parquet to uber_data
Downloaded fhvhv_tripdata_2024-05.parquet to uber_data
Downloaded fhvhv_tripdata_2024-06.parquet to uber_data
Downloaded fhvhv_tripdata_2024-07.parquet to uber_data
Downloaded fhvhv_tripdata_2024-08.parquet to uber_data
Downloaded fhvhv_tripdata_2023-01.parquet to uber_data
Downloaded fhvhv_tripdata_2023-02.parquet to uber_data
Downloaded fhvhv_tripdata_2023-03.parquet to uber_data
Downloaded fhvhv_tripdata_2023-04.parquet to uber_data


In [ ]:
def filter_uber_and_sample_monthly(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.parquet')]
    
    max_rows = 0
    for file in files:
        df = pd.read_parquet(file)
        filtered_df = df[df["hvfhs_license_num"] == "HV0003"]
        max_rows = max(max_rows, len(filtered_df))
    sample_size = calculate_sample_size(max_rows)
    print(f"Sample size for all months: {sample_size}")
    
    sampled_dataframes = []
    for file in files:
        df = pd.read_parquet(file)
        filtered_df = df[df["hvfhs_license_num"] == "HV0003"]
        sampled_df = filtered_df.sample(n=sample_size, random_state=30, replace=False)
        sampled_dataframes.append(sampled_df)

    if sampled_dataframes:
        combined_sampled_df = pd.concat(sampled_dataframes, ignore_index=True)
        return combined_sampled_df

In [349]:
sampled_uber_df = filter_uber_and_sample_monthly(uber_data_dir)

Sample size for all months: 384


/var/folders/rm/llw7py457rl4zlvsw5ymljw80000gn/T/ipykernel_32667/359863056.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_sampled_df = pd.concat(sampled_dataframes, ignore_index=True)


In [ ]:
sampled_uber_df.head()

In [217]:
sampled_uber_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21504 entries, 0 to 21503
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     21504 non-null  object        
 1   dispatching_base_num  21504 non-null  object        
 2   originating_base_num  15516 non-null  object        
 3   request_datetime      21504 non-null  datetime64[us]
 4   on_scene_datetime     15519 non-null  datetime64[us]
 5   pickup_datetime       21504 non-null  datetime64[us]
 6   dropoff_datetime      21504 non-null  datetime64[us]
 7   PULocationID          21504 non-null  int64         
 8   DOLocationID          21504 non-null  int64         
 9   trip_miles            21504 non-null  float64       
 10  trip_time             21504 non-null  int64         
 11  base_passenger_fare   21504 non-null  float64       
 12  tolls                 21504 non-null  float64       
 13  bcf             

In [ ]:
def get_and_clean_uber_data(df):
    fare_columns = ["base_passenger_fare", "tolls", "bcf", "sales_tax", "congestion_surcharge", "airport_fee"]
    columns_to_keep = ["request_datetime", "on_scene_datetime", "pickup_datetime", "dropoff_datetime",
                       "pickup_lat", "pickup_lon", "dropoff_lat", "dropoff_lon", "trip_miles",
                       "total_fare", "tips"]
    # filter uber data
    df = df[df["hvfhs_license_num"] == "HV0003"]

    # convert LocationID to coordinates
    df[["pickup_lat", "pickup_lon"]] = df["PULocationID"].map(ID_COORDS_DICT).apply(pd.Series)
    df[["dropoff_lat", "dropoff_lon"]] = df["DOLocationID"].map(ID_COORDS_DICT).apply(pd.Series)
    
    # remove invalid locations & 0 mile trips
    df = df.dropna(subset=["pickup_lat", "pickup_lon", "dropoff_lat", "dropoff_lon"])
    df = df[df["trip_miles"] != 0]
    
    # filter trips within the bounding box
    ((min_lat, min_lon), (max_lat, max_lon)) = NEW_YORK_BOX_COORDS
    pickup_in_box = (
        (df["pickup_lat"] >= min_lat) & (df["pickup_lat"] <= max_lat) &
        (df["pickup_lon"] >= min_lon) & (df["pickup_lon"] <= max_lon)
    )
    dropoff_in_box = (
        (df["dropoff_lat"] >= min_lat) & (df["dropoff_lat"] <= max_lat) &
        (df["dropoff_lon"] >= min_lon) & (df["dropoff_lon"] <= max_lon)
    )
    df = df[pickup_in_box & dropoff_in_box]
        
    # compute total fare
    df[fare_columns] = df[fare_columns].fillna(0)
    df["total_fare"] = df[fare_columns].sum(axis=1)

    df = df[columns_to_keep]
    
    return df

In [352]:
uber_data = get_and_clean_uber_data(sampled_uber_df)

In [353]:
uber_data.head(10)

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,trip_miles,total_fare,tips
0,2021-03-25 09:31:19,2021-03-25 09:32:12,2021-03-25 09:34:06,2021-03-25 09:39:19,40.620924,-73.956824,40.620924,-73.956824,1.05,8.85,0.0
1,2021-03-24 12:18:27,2021-03-24 12:23:05,2021-03-24 12:24:54,2021-03-24 12:53:16,40.763352,-73.868395,40.664003,-73.910258,11.59,34.59,0.0
2,2021-03-08 12:23:10,2021-03-08 12:25:52,2021-03-08 12:25:52,2021-03-08 12:32:20,40.775932,-73.946510,40.756729,-73.965146,2.19,18.40,0.0
3,2021-03-04 22:14:05,2021-03-04 22:19:17,2021-03-04 22:19:38,2021-03-04 22:25:14,40.685634,-73.986114,40.685634,-73.986114,0.83,8.45,3.0
4,2021-03-16 05:03:32,2021-03-16 05:10:15,2021-03-16 05:12:15,2021-03-16 05:26:09,40.833990,-73.885900,40.807347,-73.916822,2.71,16.80,0.0
5,2021-03-04 16:27:53,2021-03-04 16:30:52,2021-03-04 16:32:20,2021-03-04 16:43:20,40.841708,-73.941399,40.837827,-73.926158,2.28,10.83,0.0
6,2021-03-18 20:56:27,2021-03-18 21:00:19,2021-03-18 21:01:06,2021-03-18 21:13:02,40.740337,-73.990458,40.756729,-73.965146,2.54,18.02,0.0
7,2021-03-13 00:35:54,2021-03-13 00:38:16,2021-03-13 00:40:16,2021-03-13 00:49:16,40.640590,-73.976199,40.641886,-74.004653,1.39,6.29,1.0
8,2021-03-17 15:38:12,2021-03-17 15:41:44,2021-03-17 15:43:26,2021-03-17 16:24:06,40.620924,-73.956824,40.766238,-73.995135,12.48,61.31,0.0
9,2021-03-13 12:23:41,2021-03-13 12:26:47,2021-03-13 12:27:54,2021-03-13 12:45:38,40.612218,-73.995259,40.580922,-73.961217,4.08,20.27,3.0


In [354]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20650 entries, 0 to 21503
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   request_datetime   20650 non-null  datetime64[us]
 1   on_scene_datetime  20650 non-null  datetime64[us]
 2   pickup_datetime    20650 non-null  datetime64[us]
 3   dropoff_datetime   20650 non-null  datetime64[us]
 4   pickup_lat         20650 non-null  float64       
 5   pickup_lon         20650 non-null  float64       
 6   dropoff_lat        20650 non-null  float64       
 7   dropoff_lon        20650 non-null  float64       
 8   trip_miles         20650 non-null  float64       
 9   total_fare         20650 non-null  float64       
 10  tips               20650 non-null  float64       
dtypes: datetime64[us](4), float64(7)
memory usage: 1.9 MB


In [ ]:
def load_and_clean_uber_data():
    raise NotImplementedError()

In [ ]:
def get_uber_data():
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = find_parquet_urls(all_urls)
    taxi_data = get_and_clean_uber_data(all_parquet_urls)
    return taxi_data

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

In [ ]:
uber_data.info()

In [ ]:
uber_data.describe()

### Processing Weather Data
In this section, we processed the weather data and creates dataframes with hourly and daily granularity information retaining relevant information only.
*  `get_all_weather_csvs` returns the weather csv files in the directory
*  `clean_month_weather_data_hourly` takes csv files and returns a dataframe that contains hourly precipation and wind speed information
*  `clean_month_weather_data_daily` takes csv files and returns a dataframe that contains daily precipation, wind speed, and snowfall information. The function fills in values according to the data description for better data processing later
*  `load_and_clean_weather_data` concatnates all daily dataframes and all hourly dataframes into two large dataframes that contains all daily weather data and all hourly data respectively.

In [184]:
def get_all_weather_csvs(directory):
    weather_csvs = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.csv')]
    return weather_csvs

In [304]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file, low_memory=False)
    
    df["date"] = pd.to_datetime(df["DATE"])
    df["HourlyPrecipitation"] = (df["HourlyPrecipitation"]
        .replace("T", "0.005")  # Replace 'T' (trace) with a small float
        .str.extract(r"([\d\.]+)")  # Extract numeric part, ignore non-numeric
        .astype(float)  # Convert to float
    )

    columns = ["date", "HourlyPrecipitation", "HourlyWindSpeed"]
    df = df[columns]
    
    df = df.dropna(subset=["date"])
    df.fillna(0, inplace=True)
    df.columns = df.columns.str.lower()

    return df

In [324]:
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(csv_file, low_memory=False)
    df["date"] = pd.to_datetime(df["DATE"])

    df["DailyPrecipitation"] = (df["DailyPrecipitation"]
            .replace("T", "0.005")  
            .str.extract(r"([\d\.]+)")  
            .astype(float)  
    )
    df["DailySnowfall"] = (df["DailySnowfall"]
            .replace("T", "0.005")  
            .str.extract(r"([\d\.]+)")  
            .astype(float) 
    )
    df["DailySnowDepth"] = (df["DailySnowDepth"].replace("T", "0.005").astype(float))
    
    columns = ["date", "DailyPrecipitation", "DailyAverageWindSpeed",
               "DailySnowfall", "DailySnowDepth"]
    df = df[columns]

    df = df.dropna(subset=["date","DailyPrecipitation", "DailyAverageWindSpeed",
               "DailySnowfall", "DailySnowDepth"])
    df.columns = df.columns.str.lower()

    return df

In [325]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [326]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [327]:
hourly_weather_data.head()

,date,hourlyprecipitation,hourlywindspeed
0,2020-01-01 00:51:00,0.0,8.0
1,2020-01-01 01:51:00,0.0,8.0
2,2020-01-01 02:51:00,0.0,14.0
3,2020-01-01 03:51:00,0.0,11.0
4,2020-01-01 04:51:00,0.0,6.0


In [318]:
hourly_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56098 entries, 0 to 11638
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 56098 non-null  datetime64[ns]
 1   hourlyprecipitation  56098 non-null  float64       
 2   hourlywindspeed      56098 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.7 MB


In [319]:
hourly_weather_data.describe()

,date,hourlyprecipitation,hourlywindspeed
count,56098,56098.000000,56098.000000
mean,2022-05-29 21:14:19.618881024,0.010841,4.537238
min,2020-01-01 00:51:00,0.000000,0.000000
25%,2021-03-18 19:01:45,0.000000,0.000000
50%,2022-05-28 01:21:00,0.000000,5.000000
75%,2023-08-15 05:39:00,0.000000,7.000000
max,2024-10-22 18:51:00,3.470000,2237.000000
std,NaN,0.056735,13.883208


In [323]:
daily_weather_data.tail(10)

,date,dailyprecipitation,dailyaveragewindspeed,dailysnowfall,dailysnowdepth
11335,2022-12-22 23:59:00,0.23,8.6,0.000,0.0
11389,2022-12-23 23:59:00,1.83,10.0,0.005,0.0
11414,2022-12-24 23:59:00,0.00,11.3,0.000,0.0
11439,2022-12-25 23:59:00,0.00,8.5,0.000,0.0
11464,2022-12-26 23:59:00,0.00,7.1,0.000,0.0
11489,2022-12-27 23:59:00,0.00,5.5,0.000,0.0
11516,2022-12-28 23:59:00,0.00,5.1,0.000,0.0
11541,2022-12-29 23:59:00,0.00,6.1,0.000,0.0
11566,2022-12-30 23:59:00,0.00,2.9,0.000,0.0
11637,2022-12-31 23:59:00,0.28,1.8,0.000,0.0


In [321]:
daily_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1692 entries, 24 to 11637
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   1692 non-null   datetime64[ns]
 1   dailyprecipitation     1692 non-null   float64       
 2   dailyaveragewindspeed  1692 non-null   float64       
 3   dailysnowfall          1692 non-null   float64       
 4   dailysnowdepth         1692 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 79.3 KB


In [322]:
daily_weather_data.describe()

,date,dailyprecipitation,dailyaveragewindspeed,dailysnowfall,dailysnowdepth
count,1692,1692.000000,1692.000000,1692.000000,1692.000000
mean,2022-06-11 16:53:28.085106176,0.145018,5.000355,0.040721,0.160195
min,2020-01-01 23:59:00,0.000000,0.600000,0.000000,0.000000
25%,2021-04-05 17:59:00,0.000000,3.200000,0.000000,0.000000
50%,2022-06-19 11:59:00,0.000000,4.600000,0.000000,0.000000
75%,2023-08-16 05:59:00,0.060000,6.325000,0.000000,0.000000
max,2024-10-21 23:59:00,7.130000,14.200000,14.800000,14.000000
std,NaN,0.420440,2.339679,0.502493,1.059881


## Part 2: Storing Cleaned Data

In [77]:
engine = db.create_engine(DATABASE_URL)

In [73]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather(
    id INTEGER PRIMARY KEY,
    date DATETIME,
    hourly_precipitation FLOAT,
    hourly_wind_speed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather(
    id INTEGER PRIMARY KEY,
    date DATETIME,
    daily_precipitation FLOAT,
    daily_average_wind_speed FLOAT,
    daily_snowfall FLOAT,
    daily_snow_depth FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips(
     tpep_pickup_datetime DATATIME, 
     tpep_dropoff_datetime DATATIME,
     trip_distance FLOAT, 
     latitude_pickup FLOAT, 
     longitude_pickup FLOAT, 
     latitude_dropoff FLOAT, 
     longitude_dropoff FLOAT, 
     fare_amount FLOAT, 
     extra FLOAT, 
     mta_tax FLOAT, 
     tip_amount FLOAT, 
     tolls_amount FLOAT, 
     improvement_surcharge FLOAT, 
     congestion_surcharge FLOAT, 
     airport_fee FLOAT, 
     total_amount FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips(
    request_datetime DATETIME,
    on_scene_datetime DATETIME,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    pickup_lat FLOAT,
    pickup_lon FLOAT,
    dropoff_lat FLOAT,
    dropoff_lon FLOAT,
    trip_miles FLOAT,
    total_fare FLOAT,
    tips FLOAT
);
"""

In [83]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [85]:
# create the tables with the schema files
with engine.connect() as connection:
    with open(DATABASE_SCHEMA_FILE, "r") as f:
        schema_sql = f.read()
    schema_stmts = [stmt.strip() for stmt in schema_sql.split(";") if stmt.strip()]
    for stmt in schema_stmts:
        connection.execute(db.text(stmt))

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)